In [2]:
import pandas as pd

In [3]:
cr = pd.read_csv('../DirtyData/simple_cross_ref.csv', index_col=0)
cr_ISH = pd.DataFrame(cr['Isaiah_chapter'])
cr_BOM = pd.DataFrame(cr['BoM_chapter'])

scripts = pd.read_excel('../DirtyData/lds-scriptures.xlsx')


## From dataset scripts, I only want verses from Isaiah or the BoM

In [4]:
filtered_scripts = scripts[(scripts['book_title'] == "Isaiah") | (scripts['volume_short_title'] == "BoM")]
filtered_scripts = filtered_scripts[['verse_title', 'book_title', 'chapter_number', 'verse_number', 'scripture_text']]

## Merging verses to Isaiah-BoM references

In [5]:
# Merge references with scripture texts
ISH = pd.merge(filtered_scripts[filtered_scripts['book_title'] == "Isaiah"], cr, how = 'left', left_on='verse_title', right_on='Isaiah_chapter')
full_cr = pd.merge(ISH, filtered_scripts, how = 'left', left_on='BoM_chapter', right_on='verse_title', suffixes=['_ISH','_BOM'])
full_cr = full_cr.drop(['Isaiah_chapter','BoM_chapter' ], axis = 1)

In [6]:
# I also want the BOM chapters that were not referenced in my dataset
unref_BOM = scripts[(~scripts['verse_title'].isin(full_cr['verse_title_BOM'])) & (scripts['volume_title'] == "Book of Mormon")]
unref_BOM = unref_BOM[['verse_title', 'book_title', 'chapter_number', 'verse_number', 'scripture_text']]
unref_BOM = unref_BOM.add_suffix('_BOM')
full_cr = pd.merge(full_cr, unref_BOM, how="outer")

In [7]:
# Ordered by Isaiah verse, and non-referenced BOM verses found at the bottom 
full_cr = full_cr.sort_values(by=['chapter_number_ISH', 'verse_number_ISH'], ignore_index=True)

In [8]:
full_cr.to_csv('../DirtyData/full_cross_ref.csv', index=False)
full_cr.to_excel('../DirtyData/full_cross_ref.xlsx', index=False)